# Lab 8

In this lab, we build a convolutional neural netowrk for classification of CIFAR-10 images. 
For more information about CIFAR-10 and CIFAR-100 datasets visit: https://www.cs.toronto.edu/~kriz/cifar.html

The first step is to import the libraries.

In [27]:
### Import required libraries

import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torchvision.datasets.utils import download_url
import tarfile
from torchvision.datasets import CIFAR10
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler


### if you get trouble downloading the CIFAR-10 dataset from pytorch due to SSL issues, uncomment the following two lines
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context


Noe we set up the computing environment.

In [28]:
### Computing Environment Setup 
if torch.cuda.is_available():
    num_GPUs = torch.cuda.device_count()
    print(f'Available GPUs: {num_GPUs}')
    os.environ["CUDA_VISIBLE_DEVICES"] = 0 
    device = torch.device("cuda")
    torch.cuda.empty_cache()
    torch.cuda.set_device(int(configs['gpu']))
else:
    print('GPU is not available.')
    device = torch.device("cpu")
    num_GPUs = 0
print(f'Device is {device}.')

### Fix random seeds for reproducibility
SEED = 12345
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)  

### Set number of workers for loading data
# Interesting read: https://discuss.pytorch.org/t/guidelines-for-assigning-num-workers-to-dataloader/813/5
if num_GPUs == 0:
    num_workers = 8
else:
    num_workers = 4 * num_GPUs 



GPU is not available.
Device is cpu.


Setting up the hyperparameters of our model.

In [29]:
### Parameters and Hyperparameters Setup
batch_size = 8 
valid_size = 0.1
num_epochs = 20
learning_rate = 0.0001
momentum = 0.9
print_every_other = 2000 # print training loss every n batch
save_model = True # save the trained model
model_save_path = './cifar_net.pth' # path and file name to save the trained model


Loading the dataset using torchvision.

In [30]:
### Loading the data

train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # Interesting read: https://stackoverflow.com/questions/65676151/how-does-torchvision-transforms-normalize-operate
    ])

valid_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), 
    ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
validset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=valid_transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=valid_transform)

num_train = len(trainset)
split = int(np.floor(valid_size * num_train))
indices = list(range(num_train))
np.random.shuffle(indices)


train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)




trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=False, sampler=train_sampler, num_workers=num_workers)

validloader = torch.utils.data.DataLoader(validset, batch_size=batch_size,
                                          shuffle=False, sampler=valid_sampler, num_workers=num_workers)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=num_workers)

print(len(trainloader.sampler), len(validloader.sampler))

target_names = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
45000 5000


Let's see some samples from the training dataset.

In [31]:
### Visualization of images
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

## Select a batch of images
dataiter = iter(trainloader)
images, labels = next(dataiter)

## Show images and labels
imshow(torchvision.utils.make_grid(images))
print(' '.join(f'{target_names[labels[j]]:5s}' for j in range(batch_size))) 

TypeError: 'int' object is not callable

It iss time to build our network.

In [44]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Need to set up the loss function and optimizer.

In [45]:
### Setting up the training environment
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

In [46]:
def model_validation(net, validloader, criterion):
    correct = 0
    total = 0
    counter = 0
    total_loss = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in validloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            loss = criterion(outputs, labels).item()
            total_loss += loss
            counter+=1
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        acc = correct / total
        loss = total_loss/counter
    return loss, acc 


In [48]:
### Training 
print('Training ...')
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0
    running_acc = 0
    count_samples = 0
    
    for iter, data in enumerate(trainloader, 0):
        inputs, labels = data # data is a list of [inputs, labels]
        ## Copying the data to device
        inputs = inputs.to(device) 
        labels = labels.to(device)
        ## Zero-ing the grads
        optimizer.zero_grad()

        ## Feed forward
        outputs = net(inputs)
        ## Compupting loss
        loss = criterion(outputs, labels)
        ## Compute the gradients
        loss.backward()
        ## Perform a single optimization step
        optimizer.step()



        # Collect and print training statistics
        running_loss += loss.item()
        if iter % print_every_other == print_every_other-1:    # print every 2000 mini-batches
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()   
            count_samples = len(labels)
            print(f' Epoch: {epoch + 1}, Iteration: {iter + 1:5d}, Train Loss: {running_loss / print_every_other:.3f}, Train Accuracy: {correct / count_samples}')
            running_loss = 0.0


    valid_loss, valid_acc = model_validation(net, validloader, criterion)
    print(f' Epoch: {epoch + 1}, Validation Loss: {valid_loss}, Validation Accuracy: {valid_acc}')


print('Finished Training')

Training ...
 Epoch: 1, Iteration:  2000, Train Loss: 2.070, Train Accuracy: 0.25
 Epoch: 1, Iteration:  4000, Train Loss: 1.948, Train Accuracy: 0.375


ValueError: Expected input batch_size (4) to match target batch_size (8).

In [ ]:
### Save the trained model
if save_model:
    torch.save(net.state_dict(), model_save_path)

In [11]:
if os.path.isfile(model_save_path):
    net = Net()
    net.load_state_dict(torch.load(model_save_path))
    print('Loaded saved model.')
else:
    print('Saved model does not exist!')

Loaded saved model.


In [16]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # print(total,correct)

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

4 0
8 0
12 0
16 0
20 0
24 0
28 0
32 0
36 1
40 1
44 1
48 1
52 1
56 1
60 1
64 2
68 2
72 3
76 3
80 3
84 3
88 6
92 7
96 7
100 7
104 9
108 9
112 9
116 10
120 11
124 11
128 12
132 12
136 12
140 13
144 15
148 15
152 15
156 15
160 16
164 17
168 17
172 17
176 17
180 17
184 19
188 20
192 20
196 20
200 20
204 20
208 20
212 20
216 20
220 20
224 20
228 20
232 20
236 20
240 20
244 20
248 20
252 22
256 22
260 22
264 22
268 23
272 25
276 25
280 26
284 26
288 26
292 26
296 26
300 26
304 26
308 26
312 26
316 26
320 26
324 29
328 29
332 29
336 29
340 29
344 31
348 32
352 32
356 33
360 33
364 34
368 35
372 35
376 36
380 36
384 36
388 37
392 39
396 39
400 41
404 41
408 41
412 41
416 41
420 41
424 42
428 43
432 44
436 44
440 46
444 46
448 46
452 48
456 48
460 49
464 49
468 50
472 50
476 50
480 50
484 50
488 50
492 50
496 50
500 51
504 51
508 51
512 51
516 52
520 52
524 52
528 53
532 53
536 54
540 54
544 54
548 54
552 56
556 57
560 57
564 57
568 58
572 58
576 58
580 58
584 58
588 58
592 58
596 59
600 60
604 

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in target_names}
total_pred = {classname: 0 for classname in target_names}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[target_names[label]] += 1
            total_pred[target_names[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

## Assignment 5

In this assignment, we Use the convolutional neural network dicsussed in Lab 5 to address the following questions.

1. Implement k-fold cross-validation with k=5 and report the results including average and standard deviaion of accuracy, f1-score, sensitivity, specificity as $\%xx.x\pm xx.x$ e.g. $\% 65.1\pm 2.1$

2. Plot the training and validation loss and accuracy curves (for each epoch) in a single figure and discuss your observation with respect to the training performance. Do you observe under-fitting and/or over-fitting? If so, define in wich epochs. 

3. Implament two regularization methods to prevent over-fitting in training the model. Describe how these approaches can reduce over-fitting.
Report the results according to 1.

4. Propose, describe, and implement two approaches (not including regularization) to enhance the classificaton performance fo the model. Report the results according to 1.